# Sentiment Analysis with OpenAI and DuckDb

In [ ]:
import openai
import json
import pandas as pd

In [ ]:
# https://www.goodreads.com/book/show/22085568-the-culture-map?from_search=true&from_srp=true&qid=JOGX5h8YsY&rank=1

reviews = pd.read_csv("reviews.csv")['review'].tolist()
reviews

In [ ]:
def analyse_reviews(user_input):
    prompt = f"""
    {user_input}
    Analyse the sentiment of the reviews above and return as JSON array the result.
Provide sentiment on a scale of say 0-100?
The JSON must have these fields: sentiment, sentiment_score.
    """

    schema = {
      "type": "object",
      "properties": {
        "sentiments": {
          "type": "array",
          "items": {
            "type": "object",
            "properties": {
              "sentiment": {"type": "string", "description": "Sentiment of the review"},
              "sentiment_score": {"type": "integer","description": "Score between 1-100 of the sentiment"}
            }
          }
        }
      }
    }

    completion = openai.ChatCompletion.create(
        model="gpt-3.5-turbo",
        messages=[
            {"role": "system", "content": "You are a helpful review analysis tool."},
            {"role": "user", "content": prompt},
        ],
        functions=[{"name": "set_sentiment", "parameters": schema}],
        function_call={"name": "set_sentiment"},
    )

    try:
        generated_text = completion.choices[0].message.function_call.arguments
        return json.loads(generated_text)
    except Exception as e:
        print(f"An error occurred: {e}")
        return None

In [66]:
sentiments = analyse_reviews(reviews)

In [67]:
sentiments["sentiments"]

[{'sentiment': 'negative', 'sentiment_score': 30},
 {'sentiment': 'negative', 'sentiment_score': 20},
 {'sentiment': 'positive', 'sentiment_score': 80},
 {'sentiment': 'positive', 'sentiment_score': 90},
 {'sentiment': 'positive', 'sentiment_score': 75},
 {'sentiment': 'neutral', 'sentiment_score': 50}]

In [46]:
import pandas as pd
pd.set_option('display.max_colwidth', 75)
pd.DataFrame([
    {"review": review, **sentiment} 
    for review, sentiment in zip(reviews, sentiments["sentiments"])
])

,review,sentiment,sentiment_score
0,Possibly the worst book I've ever read.It's a huge collection of biases...,negative,10
1,"A book full of oversimplifications, generalisations and self-contradict...",negative,20
2,"I had it on my recommendations list for a long time, but my impression ...",positive,80
3,Candidate for the best book I have read in 2016 unless another one can ...,positive,90
4,A practical and comprehensive guide to how different cultures should be...,positive,70
5,The book was OK. It offers a good overview of differences between cultu...,neutral,50


In [54]:
def analyse_reviews2(user_input):
    prompt = f"""
    {user_input}
    Analyse the sentiment of the reviews above and return a JSON array as the result.
Provide sentiment on a scale of say 0-100?
The JSON must have these fields: sentiment, sentiment_score.
    """

    completion = openai.ChatCompletion.create(
        model="gpt-3.5-turbo",
        messages=[
            {"role": "system", "content": "You are a helpful review analysis tool."},
            {"role": "user", "content": prompt},
        ]
    )

    try:
        generated_text = completion.choices[0].message.content
        return json.loads(generated_text)
    except Exception as e:
        print(f"An error occurred: {e}")
        return None

In [55]:
result = analyse_reviews2(reviews[:3])

In [56]:
result

{'reviews': [{'review': "Possibly the worst book I've ever read.It's a huge collection of biases for all the possible countries and cultures. The whole book is structured with examples like: if you are working with Chinese people, you should take this approach, instead if your team is composed by German people you should do this etc....",
   'sentiment': 'negative',
   'sentiment_score': 15},
  {'review': 'A book full of oversimplifications, generalisations and self-contradiction. Plus many of the examples felt simply made up. Although it had one or two good ideas thrown in there, I am honestly not sure if this book can hardly help anyone.',
   'sentiment': 'negative',
   'sentiment_score': 20},
  {'review': 'I had it on my recommendations list for a long time, but my impression was always like: "damn, I don\'t need a book on cultural differences; I\'ve worked in many international enterprises, I have been trained, I have practical experience - it would be just a waste of time". In the

In [60]:
analyse_reviews2(reviews[:3])

{'reviews': [{'sentiment': 'negative', 'sentiment_score': 20},
  {'sentiment': 'negative', 'sentiment_score': 30},
  {'sentiment': 'positive', 'sentiment_score': 80}]}

In [61]:
analyse_reviews2(reviews[:3])

{'sentiment': 'negative', 'sentiment_score': 20}

In [57]:
analyse_reviews(reviews)

{'sentiments': [{'sentiment': 'Negative', 'sentiment_score': 0},
  {'sentiment': 'Negative', 'sentiment_score': 15},
  {'sentiment': 'Positive', 'sentiment_score': 85},
  {'sentiment': 'Positive', 'sentiment_score': 90},
  {'sentiment': 'Positive', 'sentiment_score': 80},
  {'sentiment': 'Neutral', 'sentiment_score': 55}]}